In [ ]:
from google.cloud import bigquery
from datetime import datetime

client = bigquery.Client()


sql = """
   UPDATE  sns.tweets
     SET sentiment = @sentiment
   WHERE id = @id
   """
   
   
sentiment="positive"
id="1568948724881137667"


job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter("sentiment", "STRING", sentiment),
        bigquery.ScalarQueryParameter("id", "STRING", id)
    ]
)

query_job = client.query(sql, job_config=job_config) 
results = query_job.result()  # Wait for the job to complete.
print(results)

In [ ]:
from google.cloud import bigquery
from datetime import datetime
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer


client = bigquery.Client()

sql = """
   UPDATE  sns.tweets
     SET sentiment = @sentiment
   WHERE id = @id
   """
   
sql2 = """
SELECT  id ,text FROM `snsdisasteralert-372409.sns.tweets` WHERE sentiment is null limit 1000
"""

def analyze_sentiment(text):
    # Crée un objet SentimentIntensityAnalyzer
    analyzer = SentimentIntensityAnalyzer()
    
    # Analyse le sentiment du tweet
    sentiment = analyzer.polarity_scores(text)
    # Détermine le sentiment le plus probable
    if sentiment['compound'] >= 0.05:
        sentiment_label = 'positive'
    elif sentiment['compound'] <= -0.05:
        sentiment_label = 'negative'
    else:
        sentiment_label = 'neutral'

    # Retourne la liste des résultats
    return sentiment_label

df = client.query(sql2).to_dataframe()
i =0

for index, row in df.iterrows():
    
    sentiment = analyze_sentiment(row['text'])
    job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter("sentiment", "STRING", sentiment),
        bigquery.ScalarQueryParameter("id", "STRING", row['id'])
    ]
    )
    i+=1
    query_job = client.query(sql, job_config=job_config) 
    results = query_job.result()  # Wait for the job to complete.
    print(results,"   ", i)